In [ ]:
import sys
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from scipy.misc import imresize
%pylab

In [ ]:
trainpath = 'Add path to train data here' 
testpath = 'Add path to test data here'

In [ ]:
trainlabel = [i for i in os.listdir(trainpath) if 'mask' in i]
train = [i for i in os.listdir(trainpath) if 'mask' not in i]
test = [i for i in os.listdir(testpath)]

In [ ]:
def preprocess(img, mask):
    
    if img.shape[0] == img.shape[1] == 128:
        return img
    
    z = img
    
    if img.shape[0] > img.shape[1]:
        if not mask:
            k = np.zeros((img.shape[0], img.shape[0] - img.shape[1], img.shape[2]), dtype=img.dtype)
        else:
            k = np.zeros((img.shape[0], img.shape[0] - img.shape[1]), dtype=img.dtype)
        z = np.concatenate((img,k), axis=1)
    elif img.shape[0] < img.shape[1]:
        if not mask:
            k = np.zeros((img.shape[1] - img.shape[0], img.shape[1], img.shape[2]), dtype=img.dtype)
        else:
            k = np.zeros((img.shape[1] - img.shape[0], img.shape[1]), dtype=img.dtype)
        z = np.concatenate((img,k), axis=0)
    
    return imresize(z,(128,128))

In [ ]:
for i,fil in enumerate(train):
    img = np.asarray(Image.open(trainpath + '/' + fil).convert('L'))
    train[i] = preprocess(img,1)
    
train = np.concatenate([img_i[np.newaxis] for img_i in train], axis=0)
    

In [ ]:
for i,fil in enumerate(trainlabel):
    img = np.asarray(Image.open(trainpath + '/' + fil))
    trainlabel[i] = preprocess(img,1)

trainlabel = np.concatenate([img_i[np.newaxis] for img_i in trainlabel], axis=0)    
 

In [ ]:
for i,fil in enumerate(test):
    img = np.asarray(Image.open(testpath + '/' + fil).convert('L'))
    test[i] = preprocess(img,1)

test = np.concatenate([img_i[np.newaxis] for img_i in test], axis=0)

In [ ]:
train = train.astype(np.float32)
trainlabel = trainlabel.astype(np.float32)
test = test.astype(np.float32)

In [ ]:
trainlabel.shape

In [ ]:
train = ((train-128)/256)
test = ((test-128)/256)

In [ ]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape)
  return tf.Variable(initial)

In [ ]:
def bias_variable(shape):
  initial = tf.truncated_normal(shape)
  return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [ ]:
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
out = []

batch = 10
x = tf.placeholder(tf.float32, shape=[batch, 128,128,1])
y_ = tf.placeholder(tf.float32, shape=[batch, 128,128,1])

def convnet():
    new_height = 128
    new_width = 128
    factor = 8
    number_of_classes = 1 
    
    x_image = tf.reshape(x, [batch,128,128,1]) 
    
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 =  max_pool_2x2(h_conv2)

    W_conv3 = weight_variable([5, 5, 64, 128])
    b_conv3 = bias_variable([128])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    h_pool3 = max_pool_2x2(h_conv3)
    
    W_conv4 = weight_variable([5, 5, 64, 128])
    b_conv4 = bias_variable([64])
    output_shape = [batch, 32, 32, 64]
    act4 = tf.nn.conv2d_transpose(h_pool3, W_conv4, output_shape=output_shape, strides=[1,2,2,1], padding="SAME")+b_conv4
    
    W_conv5 = weight_variable([5, 5, 32, 64])
    b_conv5 = bias_variable([32])
    output_shape = [batch, 64, 64, 32]
    act5 = tf.nn.conv2d_transpose(act4, W_conv5, output_shape=output_shape, strides=[1,2,2,1], padding="SAME")+b_conv5
    
    W_conv6 = weight_variable([5, 5, 1, 32])
    output_shape = [batch, 128, 128, 1]
    b_conv6 = bias_variable([1])
    res = tf.nn.conv2d_transpose(act5, W_conv6, output_shape=output_shape, strides=[1,2,2,1], padding="SAME") +b_conv6
    return res

res = convnet()
logits = tf.nn.softmax_cross_entropy_with_logits(res, y_)
cross_entropy = tf.reduce_sum(logits)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    for i in range(10):
        train_ = np.reshape(train[i*batch:i*batch+batch],[batch,128,128,1])
        trainlabel_ = np.reshape(trainlabel[i*batch:i*batch+batch],[batch,128,128,1])
        lab, out ,predictions, ce, _ = sess.run([y_, res, logits, cross_entropy,train_step],feed_dict={x: train_.astype(float), y_: trainlabel_.astype(float)})
        print np.argmax(predictions)